In [1]:
%pylab inline
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import clustering as cl
import warnings
warnings.filterwarnings('ignore')


Populating the interactive namespace from numpy and matplotlib


## Load Pickel file from archetype_analysis.ipynb

In [2]:
df_teams_aa = pd.read_pickle("./df_teams_only_aa.pkl")

In [3]:
df_teams_aa.iloc[:,:-1] = (df_teams_aa.iloc[:,:-1] * 100 ).round(0).astype(int)

## Load match related data

In [4]:
df_matches_19_20 = pd.read_csv("germany_match_stats/germany-bundesliga-matches-2019-to-2020-stats.csv")
df_matches_18_19 = pd.read_csv("germany_match_stats/germany-bundesliga-matches-2018-to-2019-stats.csv")
df_matches_17_18 = pd.read_csv("germany_match_stats/germany-bundesliga-matches-2017-to-2018-stats.csv")
df_matches_16_17 = pd.read_csv("germany_match_stats/germany-bundesliga-matches-2016-to-2017-stats.csv")
df_matches_15_16 = pd.read_csv("germany_match_stats/germany-bundesliga-matches-2015-to-2016-stats.csv")

In [6]:
df_all = pd.concat([df_matches_19_20, 
                    df_matches_18_19, 
                    df_matches_17_18, 
                    df_matches_16_17, 
                    df_matches_15_16],sort=False)
df_all = df_all[df_all['status'] != 'suspended']

In [7]:
df_all.sort_values('timestamp', inplace=True)
df_all.reset_index(inplace=True)
df_all.drop('index', axis=1, inplace=True)

In [8]:
df_all[df_all['status']=='complete'].tail()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
1442,1583591400,Mar 7 2020 - 2:30pm,complete,24000.0,Freiburg,Union Berlin,1.55,0.92,1.67,0.85,...,2.35,3.35,3.25,1.34,2.05,3.70,7.00,1.80,1.91,Schwarzwald-Stadion (Freiburg im Breisgau)
1443,1583591400,Mar 7 2020 - 2:30pm,complete,58324.0,Schalke 04,Hoffenheim,1.67,1.64,1.62,1.58,...,2.30,3.65,3.15,1.19,1.62,2.55,4.55,1.51,2.35,VELTINS-Arena (Gelsenkirchen)
1444,1583602200,Mar 7 2020 - 5:30pm,complete,53877.0,Borussia M'gladbach,Borussia Dortmund,2.36,1.50,2.17,1.62,...,3.10,3.80,2.25,1.15,1.53,2.30,4.00,1.45,2.50,Stadion im BORUSSIA-PARK (Mönchengladbach)
1445,1583677800,Mar 8 2020 - 2:30pm,complete,75000.0,Bayern München,Augsburg,2.17,0.75,2.23,0.69,...,1.12,11.50,19.25,1.03,1.17,1.47,2.05,1.80,1.91,Allianz Arena (München)
1446,1583686800,Mar 8 2020 - 5:00pm,complete,21409.0,Mainz 05,Fortuna Düsseldorf,1.08,0.75,1.08,0.77,...,1.91,3.85,3.95,1.20,1.65,2.60,4.70,1.57,2.25,OPEL ARENA (Mainz)


## Append results from AA Analysis as features for home and away Team to match-related-dataset

In [9]:
int_for_test = 1437
int_for_prediction = int_for_test - 20
# int_for_test = 1457
# int_for_prediction = int_for_test - 10

In [10]:
columns=pd.Series(df_teams_aa.iloc[:,:-1].columns)
columns_h = list(columns.apply(lambda x: "ht_" + x))
columns_a = list(columns.apply(lambda x: "awt_" + x))

In [11]:
df_empty_columns = pd.DataFrame(columns=(columns_h + columns_a))
df_matches_with_aa = pd.concat([df_all,df_empty_columns], axis=1)

In [12]:
df_matches_with_aa_complete = df_matches_with_aa.copy()
df_matches_with_aa_complete = df_matches_with_aa_complete.iloc[:int_for_test,:]

In [13]:
pre_cols = [col for col in df_matches_with_aa_complete.columns if 'pre' in col]
PRE_cols = [col for col in df_matches_with_aa_complete.columns if 'Pre' in col]
aa_cols_home = [col for col in df_matches_with_aa_complete.columns if 'ht_' in col]
aa_cols_away = [col for col in df_matches_with_aa_complete.columns if 'awt_' in col]

In [14]:
for index, row in df_matches_with_aa_complete.iterrows():
    teams_aa_score_home = list(df_teams_aa[df_teams_aa['common_name']==row['home_team_name']].iloc[:,:-1].iloc[0])
    teams_aa_score_away = list(df_teams_aa[df_teams_aa['common_name']==row['away_team_name']].iloc[:,:-1].iloc[0])
    
    df_matches_with_aa_complete.at[index, aa_cols_home] = teams_aa_score_home
    df_matches_with_aa_complete.at[index, aa_cols_away] = teams_aa_score_away
    

In [15]:
df_matches_with_aa_complete['HTGDIFF'] = df_matches_with_aa_complete['home_team_goal_count'] - df_matches_with_aa_complete['away_team_goal_count']
df_matches_with_aa_complete['ATGDIFF'] = df_matches_with_aa_complete['away_team_goal_count'] - df_matches_with_aa_complete['home_team_goal_count']

## Calculate AVG goal difference for home and away team rolling 10 Games

In [16]:
def avg_goal_diff(df, avg_h_a_diff, a_h_team, a_h_goal_letter):
    """
    input: 
        df = dataframe with all results
        avg_h_a_diff = name of the new column
        a_h_team = HomeTeam or AwayTeam
        a_h_goal_letter = 'H' for home or 'A' for away
    output: 
        avg_per_team = dictionary with with team as key and columns as values with new column H/ATGDIFF
    """
    df[avg_h_a_diff] = 0
    avg_per_team = {}
    all_teams = df[a_h_team].unique()
    for t in all_teams:
        df_team = df[df[a_h_team]==t].fillna(0)
        result = df_team['{}TGDIFF'.format(a_h_goal_letter)].rolling(4).mean()
        df_team[avg_h_a_diff] = result
        avg_per_team[t] = df_team
    return avg_per_team

In [17]:
d_AVGFTHG = avg_goal_diff(df_matches_with_aa_complete, 'AVGFTHG', 'home_team_name', 'H')

In [18]:
def from_dict_value_to_df(d):
    """
    input = dictionary 
    output = dataframe as part of all the values from the dictionary
    """
    df = pd.DataFrame()
    for v in d.values():
        df = df.append(v)
    return df

In [19]:
df_AVGFTHG = from_dict_value_to_df(d_AVGFTHG)
df_AVGFTHG.sort_index(inplace=True)

In [20]:
d_AVGFTAG = avg_goal_diff(df_AVGFTHG, 'AVGFTAG', 'away_team_name', 'A')
df_all = from_dict_value_to_df(d_AVGFTAG)
df_all.sort_index(inplace=True)
df_all['AVGFTAG'].fillna(0, inplace=True)

## Add per match game results from last three games

In [21]:
df_all['goal_diff'] = df_all['home_team_goal_count'] - df_all['away_team_goal_count']

for index, row in df_all[df_all['status']=='complete'].iterrows():
    if df_all['goal_diff'][index] > 0:
        df_all.at[index,'result'] = 3
    elif df_all['goal_diff'][index] == 0:
        df_all.at[index,'result'] = 2
    else:
        df_all.at[index,'result'] = 1

In [22]:
def previous_data(df, h_or_a_team, column, letter, past_n):
    """
    input: 
        df = dataframe with all results
        a_h_team = HomeTeam or AwayTeam
        column = column selected to get previous data from
    output:
        team_with_past_dict = dictionary with team as a key and columns as values with new 
                              columns with past value
    """
    d = dict()
    team_with_past_dict = dict()
    all_teams = df[h_or_a_team].unique()
    for team in all_teams:
        n_games = len(df[df[h_or_a_team]==team])
        team_with_past_dict[team] = df[df[h_or_a_team]==team]
        for i in range(1, past_n):
            d[i] = team_with_past_dict[team].assign(
                result=team_with_past_dict[team].groupby(h_or_a_team)[column].shift(i)
            ).fillna({'{}_X'.format(column): 0})
            team_with_past_dict[team]['{}_{}_{}'.format(letter, column, i)] = d[i].result
    return team_with_past_dict

In [23]:
def previous_data_call(df, side, column, letter, iterations):
    d = previous_data(df, side, column, letter, iterations)
    df_result= from_dict_value_to_df(d)
    df_result.sort_index(inplace=True)
    return df_result

In [24]:
df_last_home_results = previous_data_call(df_all, 'home_team_name', 'result', 'H', 3)
df_last_away_results = previous_data_call(df_last_home_results, 'away_team_name', 'result', 'A', 3)

In [25]:
df_last_last_HTGDIFF_results = previous_data_call(df_last_away_results, 'home_team_name', 'HTGDIFF', 'H', 3)
df_last_last_ATGDIFF_results = previous_data_call(df_last_last_HTGDIFF_results, 'away_team_name', 'ATGDIFF', 'A', 3)

In [26]:
df_last_AVGFTHG_results = previous_data_call(df_last_last_ATGDIFF_results, 'home_team_name', 'AVGFTHG', 'H', 2)
df_last_AVGFTAG_results = previous_data_call(df_last_AVGFTHG_results, 'away_team_name', 'AVGFTAG', 'A', 2)

In [27]:
df_all = df_last_AVGFTAG_results.copy()

In [28]:
df_matches_with_aa_numeric = df_all._get_numeric_data()
df_matches_with_aa_numeric.drop(['timestamp', 'goal_diff', 'result', 'home_team_goal_count', 'away_team_goal_count'], axis=1, inplace=True)
df_matches_with_aa_numeric.isnull().sum(axis = 0)

attendance               0
Pre-Match PPG (Home)     0
Pre-Match PPG (Away)     0
home_ppg                 0
away_ppg                 0
                        ..
H_HTGDIFF_2             48
A_ATGDIFF_1             24
A_ATGDIFF_2             48
H_AVGFTHG_1             24
A_AVGFTAG_1             24
Length: 68, dtype: int64

In [30]:
df_norm = (df_matches_with_aa_numeric - df_matches_with_aa_numeric.min()) / (df_matches_with_aa_numeric.max() - df_matches_with_aa_numeric.min())

In [31]:
df_next_games = df_norm.iloc[int_for_prediction:,:]
a = df_next_games.loc[:, (df_next_games != 0).any(axis=0)]
a.dropna(axis=1,inplace=True)
a.drop(['HTGDIFF', 'ATGDIFF'], inplace=True, axis=1)

In [32]:
df_next_games_teams = df_matches_with_aa_complete.iloc[int_for_prediction:,:][['home_team_name', 'away_team_name']]


In [33]:
predictable_columns = [
       'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg',
       'average_goals_per_match_pre_match', 'btts_percentage_pre_match',
       'over_15_percentage_pre_match', 'over_25_percentage_pre_match',
       'over_35_percentage_pre_match', 'over_45_percentage_pre_match',
       'over_15_HT_FHG_percentage_pre_match',
       'over_05_HT_FHG_percentage_pre_match',
       'average_corners_per_match_pre_match',
       'average_cards_per_match_pre_match', 'odds_ft_home_team_win',
       'odds_ft_draw', 'odds_ft_away_team_win', 'odds_ft_over15',
       'odds_ft_over25', 'odds_ft_over35', 'odds_ft_over45', 'odds_btts_yes',
       'odds_btts_no', 'ht_0', 'ht_1', 'ht_2', 'ht_3', 'ht_4', 'awt_0',
       'awt_1', 'awt_2', 'awt_3', 'awt_4', 'AVGFTHG', 'AVGFTAG', 'H_result_1',
       'H_result_2', 'A_result_1', 'A_result_2', 'H_HTGDIFF_1', 'H_HTGDIFF_2',
       'A_ATGDIFF_1', 'A_ATGDIFF_2', 'H_AVGFTHG_1', 'A_AVGFTAG_1'
]

In [34]:
df_X = df_norm[predictable_columns]
df_X.isnull().sum(axis = 0)

Pre-Match PPG (Home)                    0
Pre-Match PPG (Away)                    0
home_ppg                                0
away_ppg                                0
average_goals_per_match_pre_match       0
btts_percentage_pre_match               0
over_15_percentage_pre_match            0
over_25_percentage_pre_match            0
over_35_percentage_pre_match            0
over_45_percentage_pre_match            0
over_15_HT_FHG_percentage_pre_match     0
over_05_HT_FHG_percentage_pre_match     0
average_corners_per_match_pre_match     0
average_cards_per_match_pre_match       0
odds_ft_home_team_win                   0
odds_ft_draw                            0
odds_ft_away_team_win                   0
odds_ft_over15                          0
odds_ft_over25                          0
odds_ft_over35                          0
odds_ft_over45                          0
odds_btts_yes                           0
odds_btts_no                            0
ht_0                              

In [35]:
df_X.fillna(0,inplace=True)

In [36]:
X = df_X.iloc[:int_for_prediction,:]
Y = df_all.iloc[:int_for_prediction,:]['result']
Z = df_X.iloc[int_for_prediction:,:]

In [37]:
X = X.iloc[17:,:]

In [38]:
Y = Y[17:]

In [40]:
X.to_pickle("pickle_files/X.pkl")
Y.to_pickle("pickle_files/Y.pkl")
Z.to_pickle("pickle_files/Z.pkl")
df_next_games_teams.to_pickle("pickle_files/next_games.pkl")